# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [ ]:
import riak

In [ ]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

In [ ]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

## Insertar valores

In [ ]:
bucket = cliente.bucket("bucket_name")

In [ ]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [ ]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [ ]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [ ]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [ ]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [ ]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

In [ ]:
!curl http://localhost:8098/riak/bucket_name/key_one

In [ ]:
!curl http://localhost:8098/riak/bucket_name/key_two

In [ ]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

http://localhost:8098/riak/images/image_key

## Índices

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [ ]:
import uuid
import random

bucket = cliente.bucket("ejemplo_indices")

colA = ["Madrid", "Madrid", "Barcelona", "Sevilla", "Madrid"]

for i in range(0, 5):
    key = str(i+1)
    
    valor = {"value" : random.randint(0,255),
             "key_aux" :  uuid.uuid4().hex,
             "ciudad" : colA[i]}

    riak_key = bucket.new(key, data = valor)
    riak_key.add_index('idx_ciudad_bin', valor["ciudad"])
    riak_key.add_index('idx_key_aux_bin', valor["key_aux"])
    riak_key.store()
    
    print("key", key, ":", valor)


Ahora vamos a localizar las claves de "Madrid" a través del ínidice

In [ ]:
keys = bucket.get_index("idx_ciudad_bin", 'Madrid')
for key in keys.results:
    print(bucket.get(key).data)

In [ ]:
keys = bucket.get_index("idx_key_aux_bin", '5daa7aef7bad4157abae40abd2d30e08')
for key in keys.results:
    print(bucket.get(key).data)

## Clave única

In [ ]:
import time
import uuid

bucket_estado = cliente.bucket('status')

post_uuid = uuid.uuid1().hex
estado_object = riak.RiakObject(cliente, bucket_estado, post_uuid)

# Añadimos los datos que queremos almacenar.
estado_object.data = {
    'message': 'Message',
    'created': time.time(),
    'is_public': True
}

# Establecemos el content type.
estado_object.content_type = 'application/json'

# Queremos que se realice un enconding JSON sobre el valor
estado_object._encode_data = True

# Again, make sure you save it.
estado_object.store()

print(post_uuid)

In [ ]:
!curl http://localhost:8098/riak/status/d7bdfe0070cd11eaa89f080027bcdca4


http://localhost:8098/riak/status/d7bdfe0070cd11eaa89f080027bcdca4

## Listado de Buckets

In [ ]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

## Listado de Claves de un Bucket

In [ ]:
for keys in bucket_estado.stream_keys():
    for key in keys:
        print('Key %s' % key )